# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import Lasso
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:

l1_penalty = 5e2
model_all=Lasso(alpha=l1_penalty,normalize=True)
model_all.fit(sales[all_features],sales['price'])
print(model_all.intercept_)
model_all.coef_

-218136.2140351407


array([    0.        ,     0.        ,     0.        ,   134.43931396,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        , 24750.00458561,     0.        ,
       61749.10309071,     0.        ,     0.        ,    -0.        ,
           0.        ])

Find what features had non-zero weight.

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [6]:
testing = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('kc_house_valid_data.csv', dtype=dtype_dict)

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [7]:


testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [8]:
penalty_list=np.logspace(1, 7, num=13)
list1={}
for l1_penalty in (penalty_list):
    model_all=Lasso(alpha=l1_penalty,normalize=True)
    model_all.fit(training[all_features],training['price'])
    predict_v= model_all.predict(validation[all_features])
    rss_v=validation['price']-predict_v
    rss_sq=rss_v*rss_v
    rss=sum(rss_sq)
    list1[l1_penalty]=rss
    print(rss,l1_penalty)
   # list1.append(rss)
print(min(list1.items(),key=lambda x: x[1])) 
    

398213327300134.5 10.0
399041900253348.7 31.622776601683793
429791604072558.4 100.0
463739831045119.5 316.22776601683796
645898733633803.0 1000.0
1222506859427156.8 3162.2776601683795
1222506859427156.8 10000.0
1222506859427156.8 31622.776601683792
1222506859427156.8 100000.0
1222506859427156.8 316227.7660168379
1222506859427156.8 1000000.0
1222506859427156.8 3162277.6601683795
1222506859427156.8 10000000.0
(10.0, 398213327300134.5)


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [9]:
l1_penalty = 10.0
model_all=Lasso(alpha=l1_penalty,normalize=True)
model_all.fit(training[all_features],training['price'])

np.count_nonzero(model_all.coef_) + np.count_nonzero(model_all.intercept_)

15

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [10]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [11]:
def count_zero(list):
    count=0
    for i in range(17):
        if(list[i]>0.0):
            #print(list[i])
            count+=1
        if(list[i]<0.0):
            count+=1 
    return count

In [12]:
coef_dict={}
for l1_penalty in np.logspace(1, 4, num=20):
    model_all=Lasso(alpha=l1_penalty,normalize=True)
    model_all.fit(training[all_features],training['price'])
   # print(model_all.coef_)
   # count_w=count_zero(model_all.coef_)
    coef_dict[l1_penalty]=np.count_nonzero(model_all.coef_)
    print("penalty",l1_penalty,"coef",np.count_nonzero(model_all.coef_))
#print(coef_dict) 

penalty 10.0 coef 14
penalty 14.38449888287663 coef 14
penalty 20.6913808111479 coef 14
penalty 29.76351441631318 coef 14
penalty 42.81332398719393 coef 12
penalty 61.58482110660264 coef 11
penalty 88.58667904100822 coef 10
penalty 127.42749857031335 coef 9
penalty 183.29807108324357 coef 6
penalty 263.6650898730358 coef 5
penalty 379.26901907322497 coef 5
penalty 545.5594781168514 coef 5
penalty 784.7599703514607 coef 4
penalty 1128.8378916846884 coef 2
penalty 1623.776739188721 coef 2
penalty 2335.7214690901214 coef 1
penalty 3359.818286283781 coef 0
penalty 4832.930238571752 coef 0
penalty 6951.927961775606 coef 0
penalty 10000.0 coef 0


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [13]:
l1_penalty_min = 127
l1_penalty_max = 545

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [14]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [15]:
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model_all=Lasso(alpha=l1_penalty,normalize=True)
    model_all.fit(training[all_features],training['price'])
    predict_v= model_all.predict(validation[all_features])
    rss_v=validation['price']-predict_v
    rss_sq=rss_v*rss_v
    rss=sum(rss_sq)
    print(rss,l1_penalty,np.count_nonzero(model_all.coef_),model_all.coef_)

435280594306308.6 127.0 9 [-2.89806863e+03 -0.00000000e+00  1.63890625e+04  1.64984227e+02
  0.00000000e+00 -0.00000000e+00 -4.95090306e+01  0.00000000e+00
  4.13090230e+02  5.29180657e+05  4.24235012e+04  0.00000000e+00
  1.18489264e+05  0.00000000e+00  0.00000000e+00 -2.83144140e+03
  0.00000000e+00]
439167100557623.4 149.0 7 [-0.00000000e+00 -0.00000000e+00  1.19216405e+04  1.63205389e+02
  0.00000000e+00 -0.00000000e+00 -8.47647396e+00  0.00000000e+00
  0.00000000e+00  5.12243330e+05  4.21104104e+04  0.00000000e+00
  1.16972320e+05  0.00000000e+00  0.00000000e+00 -2.66288375e+03
  0.00000000e+00]
441629330447945.4 171.0 6 [-0.00000000e+00 -0.00000000e+00  7.45089377e+03  1.64385451e+02
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  4.94582208e+05  4.17082763e+04  0.00000000e+00
  1.14693948e+05  0.00000000e+00  0.00000000e+00 -2.50355122e+03
  0.00000000e+00]
444380248202994.25 193.0 6 [-0.00000000e+00 -0.00000000e+00  2.78520905e+03  1.65868625e

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [23]:

l1_penalty = 149
model_all=Lasso(alpha=l1_penalty,normalize=True)
model_all.fit(training[all_features],training['price'])
print(model_all.intercept_)
model_all.coef_

4514867.826623703


array([-0.00000000e+00, -0.00000000e+00,  1.19216405e+04,  1.63205389e+02,
        0.00000000e+00, -0.00000000e+00, -8.47647396e+00,  0.00000000e+00,
        0.00000000e+00,  5.12243330e+05,  4.21104104e+04,  0.00000000e+00,
        1.16972320e+05,  0.00000000e+00,  0.00000000e+00, -2.66288375e+03,
        0.00000000e+00])